## Test of inverted double pendulum control with mppi


In [ ]:
import torch
from mppi import MPPI
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from tqdm.notebook import tqdm
import os
import imageio

In [ ]:
%load_ext autoreload
%autoreload 2

# Run MPPI

In [ ]:
from double_pendulum_system import *
controller = DoublePendulumControl(dynamics=dynamics_analytic, horizon=15)
initial_state = torch.from_numpy(np.random.randn(6))
initial_state = torch.tensor([0,0,np.pi,0,np.pi,0])
state = initial_state
target = torch.tensor([0,0,0,0,0,0])
num_steps = 100
pbar = tqdm(range(num_steps))
mppi_states = [state.numpy()]
if not os.path.exists('mppi_plots'):
    os.makedirs('mppi_plots')

for i in pbar:
    
    action = controller.control_calcu(state)

    state = dynamics_analytic(state,action)
    state = state.squeeze()
    mppi_states.append(state.numpy())
    # print(state)
    error_i = np.linalg.norm((state-target)@torch.diag(torch.tensor([0., 0., 1, 0.1, 1, 0.1])))
    pbar.set_description(f'Goal Error: {error_i:.4f}')

    # --- Start plotting
    fig, ax = plt.subplots()
    ax = plt.axes(xlim=(state[0]-10, state[0]+10), ylim=(-2, 2))
    ax.set_aspect('equal')
    ax.grid()
    ax.set_xlabel('x')
    ax.set_ylabel('y')
    ax.set_title('MPPI Double Pendulum at t={:.2f}'.format(i*0.05))
    x = state[0]
    theta1 = state[2]
    theta2 = state[4]
    L1 = 0.5
    L2 = 0.5
    x1 = x + L1*torch.sin(theta1)
    y1 = L1*torch.cos(theta1)
    x2 = x1 + L2*torch.sin(theta2)
    y2 = y1 + L2*torch.cos(theta2)
    plt.plot([x,x1],[0,y1],color='black')   
    plt.plot([x1,x2],[y1,y2],color='black')
    filename = os.path.join('mppi_plots', 'mppi_plot_{:03d}.png'.format(i))
    # plt.show()
    plt.savefig(filename)
    plt.close()
    if error_i < 0.3:
        num_steps = i
        print("Goal Reached!")
        break
    # --- End plotting

images = []
for i in range(num_steps):
    filename = os.path.join('mppi_plots', 'mppi_plot_{:03d}.png'.format(i))
    images.append(imageio.imread(filename))
imageio.mimsave('mppi_double_pendulum.gif', images, duration=0.1)

# Plot MPPI

In [ ]:
# Plot phase space trajectory with three different subplots with x and y labels and subtitles
fig, axs = plt.subplots(3, 1, figsize=(10, 15))
axs[0].plot(np.array(mppi_states)[:,0], np.array(mppi_states)[:,1])
axs[0].set_title('x vs x_dot')
axs[0].set_xlabel('x')
axs[0].set_ylabel('x_dot')
axs[1].plot(np.array(mppi_states)[:,2], np.array(mppi_states)[:,3])
axs[1].set_title('theta1 vs theta1_dot')
axs[1].set_xlabel('theta1')
axs[1].set_ylabel('theta1_dot')
axs[2].plot(np.array(mppi_states)[:,4], np.array(mppi_states)[:,5])
axs[2].set_title('theta2 vs theta2_dot')
axs[2].set_xlabel('theta2')
axs[2].set_ylabel('theta2_dot')
plt.show()
#Store the figure
fig.savefig('mppi_double_pendulum_phase_space.png')

# Plot trajectory in state space in 6 subplots with x and y labels and subtitles
fig, axs = plt.subplots(3, 2, figsize=(10, 10))
axs[0, 0].plot(np.array(mppi_states)[:,0])
axs[0, 0].set_title('x')
axs[0, 1].plot(np.array(mppi_states)[:,1])
axs[0, 1].set_title('x_dot')
axs[1, 0].plot(np.array(mppi_states)[:,2])
axs[1, 0].set_title('theta1')
axs[1, 1].plot(np.array(mppi_states)[:,3])
axs[1, 1].set_title('theta1_dot')
axs[2, 0].plot(np.array(mppi_states)[:,4])
axs[2, 0].set_title('theta2')
axs[2, 1].plot(np.array(mppi_states)[:,5])
axs[2, 1].set_title('theta2_dot')
plt.show()

#Store the figure
fig.savefig('mppi_double_pendulum_state_space.png')

# Run DDP

In [ ]:
from DDPController import *
from doublePendulumDynamics import *
from double_pendulum_system import *
import autograd.numpy as np


if not os.path.exists('ddp_plots'):
    os.makedirs('ddp_plots')

state_dim = 6
action_dim = 1
x_final = np.array([.0, .0, .0, .0, .0, .0])

Q = np.diag([50., 200., 1000, 130., 1000, 130])
R = np.array([[.3]])
terminal_scale = 10.0
cost = Cost(x_final, terminal_scale, Q, R)
DDP_dynamic = dynamics_numpy

controller = DDPcontroller(DDP_dynamic, cost, T = 10, max_iter= 100)


# initial_state = np.random.randn(state_dim)
initial_state = np.array([0,0,np.pi ,0,np.pi,0])
state = initial_state
ddp_states = [initial_state]
target = x_final

num_steps = 100
pbar = tqdm(range(num_steps))

for i in pbar:
    action = controller.command(state)
    
    state = DDP_dynamic(state, action)
    state = state.squeeze()
    ddp_states.append(state)
    error_i = dp_state_def(state, target)
    error_i = np.linalg.norm(error_i @ np.diag([0.0, 0.0, 1, 0.1, 1, 0.1]))
    pbar.set_description(f'Goal Error: {error_i:.4f}')

    # --- Start plotting
    fig, ax = plt.subplots()
    ax = plt.axes(xlim=(state[0]-10, state[0]+10), ylim=(-2, 2))
    ax.set_aspect('equal')
    ax.grid()
    ax.set_xlabel('x')
    ax.set_ylabel('y')
    ax.set_title('DDP Double Pendulum at t={:.2f}'.format(i*0.05))
    x = state[0]
    theta1 = state[2]
    theta2 = state[4]
    L1 = 0.5
    L2 = 0.5
    x1 = x + L1*np.sin(theta1)
    y1 = L1*np.cos(theta1)
    x2 = x1 + L2*np.sin(theta2)
    y2 = y1 + L2*np.cos(theta2)
    plt.plot([x,x1],[0,y1],color='black')   
    plt.plot([x1,x2],[y1,y2],color='black')
    filename = os.path.join('ddp_plots', 'ddp_plot_{:03d}.png'.format(i))
    
    # plt.show()
    plt.savefig(filename)
    plt.close()
    if error_i < 0.1 and i > 50:
        num_steps = i
        break
    # --- End plotting

images = []
for i in range(num_steps):
    filename = os.path.join('ddp_plots', 'ddp_plot_{:03d}.png'.format(i))
    images.append(imageio.imread(filename))
imageio.mimsave('ddp_double_pendulum.gif', images, duration=0.1)
    # --- End plotting
plt.show()
plt.close()



# Plot DDP

In [ ]:
# Plot phase space trajectory with three different subplots with x and y labels and subtitles
fig, axs = plt.subplots(3, 1, figsize=(10, 15))
axs[0].plot(np.array(ddp_states)[:,0], np.array(ddp_states)[:,1])
axs[0].set_title('x vs x_dot')
axs[0].set_xlabel('x')
axs[0].set_ylabel('x_dot')
axs[1].plot(np.array(ddp_states)[:,2], np.array(ddp_states)[:,3])
axs[1].set_title('theta1 vs theta1_dot')
axs[1].set_xlabel('theta1')
axs[1].set_ylabel('theta1_dot')
axs[2].plot(np.array(ddp_states)[:,4], np.array(ddp_states)[:,5])
axs[2].set_title('theta2 vs theta2_dot')
axs[2].set_xlabel('theta2')
axs[2].set_ylabel('theta2_dot')
plt.show()
#Store the figure
fig.savefig('ddp_double_pendulum_phase_space.png')




# Plot trajectory in state space in 6 subplots with x and y labels and subtitles
fig, axs = plt.subplots(3, 2, figsize=(10, 10))
axs[0, 0].plot(np.array(mppi_states)[:,0])
axs[0, 0].set_title('x')
axs[0, 1].plot(np.array(mppi_states)[:,1])
axs[0, 1].set_title('x_dot')
axs[1, 0].plot(np.array(mppi_states)[:,2])
axs[1, 0].set_title('theta1')
axs[1, 1].plot(np.array(mppi_states)[:,3])
axs[1, 1].set_title('theta1_dot')
axs[2, 0].plot(np.array(mppi_states)[:,4])
axs[2, 0].set_title('theta2')
axs[2, 1].plot(np.array(mppi_states)[:,5])
axs[2, 1].set_title('theta2_dot')
plt.show()

#Store the figure
fig.savefig('mppi_double_pendulum_state_space.png')

# Plot Phase Comparision Below

In [ ]:
# Plot phase space trajectory
fig, axs = plt.subplots(3, 1, figsize=(8, 12))
axs[0].plot(np.array(ddp_states)[:,0], np.array(ddp_states)[:,1])
axs[0].plot(np.array(mppi_states)[:,0], np.array(mppi_states)[:,1])
axs[0].set_title('x vs $\dot{x}$', fontsize=14)
axs[0].set_xlabel('x', fontsize=14)
axs[0].set_ylabel('$\dot{x}$', fontsize=14)

axs[1].plot(np.array(ddp_states)[:,2], np.array(ddp_states)[:,3])
axs[1].plot(np.array(mppi_states)[:,2], np.array(mppi_states)[:,3])
axs[1].set_title(r'$\theta_1$ vs $\dot{\theta}_1$')
axs[1].set_xlabel(r'$\theta_1$')
axs[1].set_ylabel(r'$\dot{\theta_1}$')
axs[1].plot([0], [0], marker='o', markersize=5, color="red")
axs[1].grid()

axs[2].plot(np.array(ddp_states)[:,4], np.array(ddp_states)[:,5])
axs[2].plot(np.array(mppi_states)[:,4], np.array(mppi_states)[:,5])
axs[2].set_title(r'$\theta_2$ vs $\dot{\theta}_2$')
axs[2].set_xlabel(r'$\theta_2$')
axs[2].set_ylabel(r'$\dot{\theta_2}$')
axs[2].plot([0], [0], marker='o', markersize=5, color="red")
axs[2].grid()

fig.legend(['DDP', 'MPPI'], bbox_to_anchor=(0.6, 0.07), ncol=2, )
fig.subplots_adjust(hspace=0.5)

# Add title to the figure
fig.suptitle('Phase Space Trajectory Comparison for Downwards Initial State', fontsize=16)

plt.show()

#Store the figure
fig.savefig('mppi_vs_ddp_down.png')

# Plot State Versus Step below for DDP and MPPI

In [ ]:
# Plot states versus time for DDP and MPPI controller below with six subpolots each for each state
fig, axs = plt.subplots(3, 2, figsize=(10, 10))

axs[0, 0].plot(np.array(ddp_states)[:,0])
axs[0, 0].plot(np.array(mppi_states)[:,0])
axs[0, 0].set_title('x vs step')
axs[0, 0].set_xlabel('step')
axs[0, 0].set_ylabel('x')

axs[0, 1].plot(np.array(ddp_states)[:,1])
axs[0, 1].plot(np.array(mppi_states)[:,1])
axs[0, 1].set_title('$\dot{x}$ vs step')
axs[0, 1].set_xlabel('step')
axs[0, 1].set_ylabel('$\dot{x}$')

axs[1, 0].plot(np.array(ddp_states)[:,2])
axs[1, 0].plot(np.array(mppi_states)[:,2])
axs[1, 0].set_title(r'$\theta_1$ vs step')
axs[1, 0].set_xlabel('step')
axs[1, 0].set_ylabel(r'$\theta_1$')

axs[1, 1].plot(np.array(ddp_states)[:,3])
axs[1, 1].plot(np.array(mppi_states)[:,3])
axs[1, 1].set_title(r'$\dot{\theta}_1$ vs step')
axs[1, 1].set_xlabel('step')
axs[1, 1].set_ylabel(r'$\dot{\theta}_1$')

axs[2, 0].plot(np.array(ddp_states)[:,4])
axs[2, 0].plot(np.array(mppi_states)[:,4])
axs[2, 0].set_title(r'$\theta_2$ vs step')
axs[2, 0].set_xlabel('step')
axs[2, 0].set_ylabel(r'$\theta_2$')

axs[2, 1].plot(np.array(ddp_states)[:,5])
axs[2, 1].plot(np.array(mppi_states)[:,5])
axs[2, 1].set_title(r'$\dot{\theta}_2$ vs step')
axs[2, 1].set_xlabel('step')
axs[2, 1].set_ylabel(r'$\dot{\theta}_2$')

# Add title to the figure
fig.suptitle('State versus Step for Downwards Initial State', fontsize=16)
fig.legend(['DDP', 'MPPI'], bbox_to_anchor=(0.6, 0.07), ncol=2, )
fig.subplots_adjust(hspace=0.5, wspace=0.3)

# Save the figure
fig.savefig('state_compare_downward_states.png')